In [7]:
import pandas as pd

x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')

print(x_train.shape)
print('Old shape: ', y_train.shape)

df = pd.DataFrame(0, index=range(y_train.shape[0]), columns=[1])
bools = y_train.notna()
vals = y_train.notna().sum(axis=1)

for i in range(y_train.shape[0]):
    if vals.iloc[0] == 1:
        df.iloc[i, 0] = str(y_train.iloc[i, 0])
    else:
        moves = []
        for j in range(y_train.shape[1]):
            if bools.iloc[i, j]:
                moves.append(str(y_train.iloc[i, j]))
        df.iloc[i, 0] = ', '.join(moves)

y_train = df
print('New shape: ', y_train.shape)

(60, 1)
Old shape:  (60, 3)
New shape:  (60, 1)


C:\Users\arora\AppData\Local\Temp\ipykernel_19244\1278530641.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '(2, 1, 'X')' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i, 0] = str(y_train.iloc[i, 0])


In [8]:
import os
import json

with open('sys_prompt_zero.txt') as f:
    system_prompt = f.read()

with open('gpt_zero.jsonl', mode='w', encoding='utf-8') as jsonl_file:

    for i in range(x_train.shape[0]):
        messages = []

        # System prompt
        message = {
            "role": 'system',
            "content": system_prompt
        }
        messages.append(message)

        # User prompt
        message = {
            "role": 'user',
            "content": x_train.iloc[i, 0]
        }
        messages.append(message)

        # System response
        message = {
            "role": 'assistant',
            "content": y_train.iloc[i, 0]
        }
        messages.append(message)

        data = {"messages": messages}
        jsonl_file.write(json.dumps(data) + "\n")



In [9]:
import openai
model_name = 'gpt-3.5-turbo'
data_path = 'gpt_zero.jsonl'

log_file = openai.files.create(
    file=open(data_path, "rb"),
    purpose='fine-tune'
)

log_job = openai.fine_tuning.jobs.create(training_file=log_file.id, model=model_name)
